In [ ]:
     !pip install nltk gensim scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 53.0 MB/s eta 0:00:00


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
import pandas as pd
import numpy as np
import re
from ast import literal_eval

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from gensim.models import FastText

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report

import joblib


In [ ]:
df = pd.read_csv("book_details.csv")

df = df.dropna(subset=["description", "genres"]).reset_index(drop=True)

def parse_genres(g):
    try:
        return literal_eval(g)
    except:
        return []

df["genres_parsed"] = df["genres"].apply(parse_genres)

primary_genres = []
for gs in df["genres_parsed"]:
    primary_genres.append(gs[0] if len(gs) > 0 else "Unknown")

df["primary_genre"] = primary_genres


In [ ]:
df["label"] = df["primary_genre"].apply(
    lambda g: 1 if "Fiction" in g else 0
)


In [ ]:
stop_words = set(stopwords.words("english"))
lemm = WordNetLemmatizer()

def clean_text(text):
    text = text.lower()
    text = re.sub(r"http\S+|www\S+", " ", text)
    text = re.sub(r"[^a-z\s]", " ", text)
    tokens = nltk.word_tokenize(text)
    tokens = [
        lemm.lemmatize(tok)
        for tok in tokens
        if tok not in stop_words and len(tok) > 2
    ]
    return tokens


In [ ]:
df["tokens"] = df["description"].apply(clean_text)


In [ ]:
sentences = df["tokens"].tolist()

ft_model = FastText(
    sentences,
    vector_size=100,
    window=5,
    min_count=1,
    sg=1,        # SKIP-GRAM
    workers=4
)

ft_model.save("fasttext_skipgram.model")
print("FastText model saved!")


FastText model saved!


In [ ]:
def document_vector(tokens):
    vectors = [ft_model.wv[w] for w in tokens if w in ft_model.wv]
    if len(vectors) == 0:
        return np.zeros(100)
    return np.mean(vectors, axis=0)

X_vectors = df["tokens"].apply(document_vector)

X = np.vstack(X_vectors.values)
y = df["label"].values


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [ ]:
lr = LogisticRegression(max_iter=3000)
lr.fit(X_train, y_train)

pred_lr = lr.predict(X_test)
print("LR Accuracy:", accuracy_score(y_test, pred_lr))
print(classification_report(y_test, pred_lr))


LR Accuracy: 0.7147410358565737
              precision    recall  f1-score   support

           0       0.73      0.95      0.82       881
           1       0.58      0.15      0.24       374

    accuracy                           0.71      1255
   macro avg       0.65      0.55      0.53      1255
weighted avg       0.68      0.71      0.65      1255



In [ ]:
svm = LinearSVC()
svm.fit(X_train, y_train)

pred_svm = svm.predict(X_test)
print("SVM Accuracy:", accuracy_score(y_test, pred_svm))
print(classification_report(y_test, pred_svm))


SVM Accuracy: 0.7179282868525897
              precision    recall  f1-score   support

           0       0.73      0.95      0.83       881
           1       0.59      0.17      0.27       374

    accuracy                           0.72      1255
   macro avg       0.66      0.56      0.55      1255
weighted avg       0.69      0.72      0.66      1255



In [ ]:
joblib.dump(lr, "classifier.pkl")
print("Logistic Regression classifier saved!")


Logistic Regression classifier saved!


In [ ]:
joblib.dump(svm, "classifier.pkl")
print("SVM classifier saved!")


SVM classifier saved!


In [ ]:
ft_model = FastText.load("fasttext_skipgram.model")
clf = joblib.load("classifier.pkl")


In [ ]:
def predict_genre(text):
    tokens = clean_text(text)
    vec = document_vector(tokens)
    pred = clf.predict([vec])[0]
    return "Fiction" if pred == 1 else "Not Fiction"

print(predict_genre("A magical journey of a young wizard"))


Not Fiction
